# Student Alcohol Consumption
Link: https://www.kaggle.com/datasets/uciml/student-alcohol-consumption

In [15]:
import numpy as np
import pandas as pd

math_df = pd.read_csv('Datasets/Student Alcohol Consumption/student-mat.csv')
portuguese_df =  pd.read_csv('Datasets/Student Alcohol Consumption/student-por.csv')
print(np.shape(math_df))        # Shape [395, 33] 
print(np.shape(portuguese_df))  # Shape [649, 33]
# Not unreasonable to use KNN for both

(395, 33)
(649, 33)


The main goal here is to predict the student's grades (both math and portuguese) using relevant labels.

Need to parse the data set so the values can be used (i.e. yes/no should be changed to 1/0).

In [27]:
# Changing yes/no features to 1/0
yes_no_cols = [col for col in math_df.columns if set(math_df[col].unique()) == {'yes', 'no'}]
for col in yes_no_cols:
  math_df[col] = np.where(math_df[col] == 'yes', 1, 0)

print(math_df.iloc[0, :])


school             GP
sex                 F
age                18
address             U
famsize           GT3
Pstatus             A
Medu                4
Fedu                4
Mjob          at_home
Fjob          teacher
reason         course
guardian       mother
traveltime          2
studytime           2
failures            0
schoolsup           1
famsup              0
paid                0
activities          0
nursery             1
higher              1
internet            0
romantic            0
famrel              4
freetime            3
goout               4
Dalc                1
Walc                1
health              3
absences            6
G1                  5
G2                  6
G3                  6
Name: 0, dtype: object


In [17]:
perm_idx = np.random.permutation(math_df.shape[0])  # Shuffling entries around
# Getting 20% of the data set
valid_num = int(math_df.shape[0] * 0.2)
valid_idx = perm_idx[:valid_num]
train_set_idx = perm_idx[valid_num:]  # indices for the training set

